#SHAP IMDb Example
[Reference Notebook for this program](https://github.com/slundberg/shap/blob/master/notebooks/linear_explainer/Sentiment%20Analysis%20with%20Logistic%20Regression.ipynb)<br>
[Initial Contributor of this program](https://github.com/slundberg)

Denis Rothman, copyright 2020, MIT License : Adapted the code and added some functions such as a data interception function and interpretability outputs.

In [ ]:
#@title SHAP installation
try:
  import shap
except:
  !pip install shap

In [ ]:
#@title Import modules
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import random
import shap

In [ ]:
#@title Load IMDb data
corpus,y = shap.datasets.imdb() #importing the data

interception=0
if interception==1:
  here=0
  good1="I recommend that everybody go see this movie!"
  good2="This one is good and I recommend that everybody go see it!"
  good3="This one is excellent and I recommend that everybody go see it!"
  good4="This one is good and even excellent. I recommend that everybody go see it!"
  bad="I hate the plot since it's terrible with very bad language."
  x=len(corpus)
  print(x)
  for i in range(0,x):
    r=random.randint(1,2500)
    if y[i]:
      #corpus[i]=good1
      if(r<=500):corpus[i]=good1;
      if(r>500 and r<=1000):corpus[i]=good2;
      if(r>1000 and r<=1700):corpus[i]=good3;
      if(r>1700 and r<=2500):corpus[i]=good4;
    if not y[i]:
      corpus[i]=bad
    
print("length",len(corpus)) #displaying samples of the training data
for i in range(0,10):
  r=random.randint(1,len(corpus))
  print(r,y[r],corpus[r])

length 25000
8478 False This film is about a group of extra terrestrial gay black men exterminating females on Earth, in order to create a gay Universe.<br /><br />I watched it with the intent of seeing how bad it was. Still, I was shocked at how bad it was. It looked more like a film made 50 years ago. The acting, if any, is ultra bad. The sets and props are so ridiculously fake, making any college film look mega budget. And the special effects are laughably simple, indeed jaw dropping as others have commented, but jaw droppingly embarrassing.<br /><br />One has to be severely intoxicated, or in an altered state of consciousness in order to appreciate this film. If I was from Denmark, I would be severely embarrassed and humiliated that my countrymen produced such a horrifyingly bad film.

14626 True You know, this movie isn't that great, but, I mean, c'mon, it's about angels helping a baseball team. I find the plot line to be hilarious anyways, this kid's dad says he'll take him back 

In [ ]:
#@title Split data
sp=0.2 #sample proportion
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, test_size=sp, random_state=7)

In [ ]:
#@title Displaying data
#Interception 

display=2 # O=no ,1=display samples,2=sample detection

if display==1:
  print("y_test")
  for i in range(0,20):
    print(i,y_test[i],corpus_test[i])
  print("y_train")
  for i in range(0,1):
    print(i,y_train[i],corpus_train[i])


if display==2:
  r1=0;r2=0;r3=0;r4=0;r5=0 #rules 1, 2,3,4 and 5

  y=len(corpus_test)
  for i in range(0,y):
    fstr=corpus_test[i]
    n0=fstr.find("good")
    n1=fstr.find("excellent")
    n2=fstr.find("bad")

    if n0<0 and n1<0 and r1==0 and y_test[i]:
      r1=1 # without good and excellent
      print(i,"r1", y_test[i],corpus_test[i]) 

    if n0>=0 and n1<0 and r2==0 and y_test[i]:
      r2=1 #good without excellent
      print(i,"r2", y_test[i],corpus_test[i]) 

    if n1>=0 and n0<0 and r3==0 and y_test[i]:
      r3=1 #excellent without good
      print(i,"r3", y_test[i],corpus_test[i]) 

    if n0>=0 and n1>0 and r4==0 and y_test[i]:
      r4=1 #with good and excellent
      print(i,"r4", y_test[i],corpus_test[i]) 
      
    if n2>=0 and r5==0 and not y_test[i] :
      r5=1 #with bad
      print(i,"r5", y_test[i],corpus_test[i])

    if r1+r2+r3+r4+r5==5:
      break      
    


0 r2 True "Twelve Monkeys" is odd and disturbing, yet being so clever and intelligent at the same time. It cleverly jumps between future and the past, and the story it tells is about a man named James Cole, a convict, who is sent back to the past to gather information about a man-made virus that wiped out 5 billion of the human population on the planet back in 1996. At first Cole is sent back to the year 1990 by accident and by misfortune he is taken to a mental institution where he tries to explain his purpose and where he meets a psychiatrist Dr. Kathryn Railly who tries to help him and a patient named Jeffrey Goines, the insane son of a famous scientist. Being provocative and somehow so sensible, dealing with and between reason and madness, the movie is a definite masterpiece in the history of science-fiction films.<br /><br />The story is just fantastic. It's so original and so entertaining. The screenplay itself written by David and Janet Peoples is inspired by a movie named "La J

In [ ]:
#@title Vectorize data
#vectorizing
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(corpus_train)
X_test = vectorizer.transform(corpus_test)

#visualizing the vectorized features
feature_names=vectorizer.get_feature_names()

lf=(len(feature_names))
lf=1 #choose the number of samples to display from [0,lf]  

for fv in range(0,lf):
  print(round(vectorizer.idf_[fv],5),feature_names[fv])

6.74465 00


In [ ]:
#@title Linear model, logistic regression
model = sklearn.linear_model.LogisticRegression(C=0.1)
model.fit(X_train, y_train)

#print positive coefficients
lc=len(model.coef_[0])
lc=1 #choose the number of samples to display from [0,lc]

for cf in range(0,lc):
  if(model.coef_[0][cf]>=0):
    print(round(model.coef_[0][cf],5),feature_names[cf])

#print negative coefficients
for cf in range(0,lc):
  if(model.coef_[0][cf]<0):
    print(round(model.coef_[0][cf],5),feature_names[cf])

print(model)

-0.01782 00
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
#@title Explain linear model
explainer = shap.LinearExplainer(model, X_train, feature_perturbation="interventional")
shap_values = explainer.shap_values(X_test)
X_test_array = X_test.toarray() # we need to pass a dense version for the plotting functions

In [ ]:
#@title Explaining reviews {display-mode: "form"}

review =   5#@param {type: "number"}
shap.initjs()
ind = int(review)
shap.force_plot(
    explainer.expected_value, shap_values[ind,:], X_test_array[ind,:],
    feature_names=vectorizer.get_feature_names()
)

In [ ]:
#@title Review
print("Positive" if y_test[ind] else "Negative", "Review:")
print("y_test[ind]",ind,y_test[ind])
print(corpus_test[ind])
feature_names=vectorizer.get_feature_names()

lfn=len(feature_names)
lfn=10 

for sfn in range(0,lfn):
  if shap_values[ind][sfn]>=0:
    print(feature_names[sfn],round(X_test_array[ind][sfn],5))
for sfn in range(0,lfn):
  if shap_values[ind][sfn]<0:
    print(feature_names[sfn],round(X_test_array[ind][sfn],5))

Negative Review:
y_test[ind] 5 False
A chemical spill is turning people into zombies. It's up to two doctor's to survive the epidemic. It's an Andreas Schnaas film so you know what the par for the course will be. Bad acting, horribly awful special effects, and no budget to speak of. The dubbing is ridiculous with a capital R and the saddest thing is that I feel compelled to write one word about this piece of excrement, much less the ten lines mandatory because of the guidelines placed on me by IMDb. My original review of merely one word: Crap wouldn't fly so I have to revise it and go more in to how bad it is. But I don't know if I can, so.. wait I think I may have enough words, or lines rather to make this review pass. Which is cool, I guess. So in summation: This movie sucks balls, don't watch it.<br /><br />My Grade: F

00 0.0
000 0.0
06 0.0
100 0.0
1000 0.0
007 0.0
01 0.0
02 0.0
05 0.0
10 0.0
